In [1]:
pip install numpy pandas tensorflow torch


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [9]:
import numpy as np
import pandas as pd
import tensorflow as tf
import torch
import torch.nn as nn
import torch.optim as optim

data_path = '/content/indian movies.csv'
data = pd.read_csv(data_path)

print(data.head())
print(data.columns)

data['Rating(10)'] = pd.to_numeric(data['Rating(10)'].replace('-', np.nan), errors='coerce')
data['Votes'] = pd.to_numeric(data['Votes'].replace('-', np.nan).str.replace(',', ''), errors='coerce')

data.fillna(method='ffill', inplace=True)

categorical_columns = ['Genre', 'Language']
data = pd.get_dummies(data, columns=categorical_columns)

numerical_columns = ['Votes']
data[numerical_columns] = (data[numerical_columns] - data[numerical_columns].mean()) / data[numerical_columns].std()

target_column = 'Rating(10)'
X = data.drop(columns=[target_column, 'ID', 'Movie Name', 'Year', 'Timing(min)'])
y = data[target_column]


X = X.values.astype(np.float32)
y = y.values.astype(np.float32)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def create_tf_model(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(input_shape,)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return model

tf_model = create_tf_model(X_train.shape[1])
tf_model.summary()

tf_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

class PyTorchModel(nn.Module):
    def __init__(self, input_dim):
        super(PyTorchModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

input_dim = X_train.shape[1]
pytorch_model = PyTorchModel(input_dim)

criterion = nn.MSELoss()
optimizer = optim.Adam(pytorch_model.parameters(), lr=0.001)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

num_epochs = 10
batch_size = 32

for epoch in range(num_epochs):
    for i in range(0, len(X_train_tensor), batch_size):
        X_batch = X_train_tensor[i:i+batch_size]
        y_batch = y_train_tensor[i:i+batch_size]

        outputs = pytorch_model(X_batch)
        loss = criterion(outputs, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

with torch.no_grad():
    outputs = pytorch_model(X_test_tensor)
    mse_loss = criterion(outputs, y_test_tensor)
    print(f'Mean Squared Error: {mse_loss.item():.4f}')


          ID              Movie Name  Year Timing(min) Rating(10)  Votes  \
0  tt0398974             Dr. Shaitan  1960           -          -      -   
1  tt1702558              Nadir Khan  1968           -          -      -   
2  tt0493437  Apna Sapna Money Money  2006     134 min        5.3  1,892   
3  tt0273405          Aag Aur Sholay  1987           -        2.2     20   
4  tt0049595                 Parivar  1956           -        7.4     21   

                                  Genre Language  
0                                     -    hindi  
1                                     -     urdu  
2  Comedy, Musical, Romance                hindi  
3                                     -     urdu  
4     Comedy, Drama, Family                hindi  
Index(['ID', 'Movie Name', 'Year', 'Timing(min)', 'Rating(10)', 'Votes',
       'Genre', 'Language'],
      dtype='object')
Model: "sequential_1"
_________________________________________________________________
 Layer (type)            